In [41]:
import pandas as pd
import re

def addRowsToDataframe(df, row):

    df.loc[len(df)] = row



df = pd.read_json('../db_hts/chapter_1.json')
columns_df = df.columns.tolist()
final_dict = {}
main_numbers_pattern = re.compile(r'^[\d]{4}')
start = True
current_main_hts = ['']
previous_main_hts = ['']

for index,row in df.iterrows():

    current_main_hts = main_numbers_pattern.findall(row['htsno'])

    if start:
        previous_main_hts = current_main_hts
        final_dict[current_main_hts[0]] = pd.DataFrame(columns=columns_df)
        addRowsToDataframe(final_dict[current_main_hts[0]], row)

    if len(current_main_hts) == 0:
        current_main_hts = previous_main_hts

    if (previous_main_hts[0] != current_main_hts[0]) and (len(current_main_hts)):
        final_dict[current_main_hts[0]] = pd.DataFrame(columns=columns_df)
        addRowsToDataframe(final_dict[current_main_hts[0]], row)
        previous_main_hts = current_main_hts  
    elif start == False:
        addRowsToDataframe(final_dict[current_main_hts[0]], row)
    start = False

final_dict

{'0101':             htsno indent                             description superior  \
 0            0101      0  Live horses, asses, mules and hinnies:     None   
 1                      1                                 Horses:     true   
 2      0101.21.00      2               Purebred breeding animals     None   
 3   0101.21.00.10      3                                   Males     None   
 4   0101.21.00.20      3                                 Females     None   
 5      0101.29.00      2                                   Other     None   
 6   0101.29.00.10      3        Imported for immediate slaughter     None   
 7   0101.29.00.90      3                                   Other     None   
 8   0101.30.00.00      1                                   Asses     None   
 9         0101.90      1                                  Other:     None   
 10  0101.90.30.00      2        Imported for immediate slaughter     None   
 11  0101.90.40.00      2                               

In [43]:
final_dict['0101']

,htsno,indent,description,superior,units,general,special,other,footnotes,quotaQuantity,additionalDuties,addiitionalDuties
0,0101,0,"Live horses, asses, mules and hinnies:",None,[],,,,[],,,NaN
1,,1,Horses:,true,[],,,,None,None,None,NaN
2,0101.21.00,2,Purebred breeding animals,None,[],Free,,Free,"[{'columns': ['general'], 'marker': '1', 'valu...",None,None,NaN
3,0101.21.00.10,3,Males,None,[No.],,,,[],None,None,NaN
4,0101.21.00.20,3,Females,None,[No.],,,,[],None,None,NaN
5,0101.29.00,2,Other,None,[],Free,,20%,"[{'columns': ['general'], 'marker': '1', 'valu...",None,None,NaN
6,0101.29.00.10,3,Imported for immediate slaughter,None,[No.],,,,[],None,None,NaN
7,0101.29.00.90,3,Other,None,[No.],,,,[],None,None,NaN
8,0101.30.00.00,1,Asses,None,[No.],6.8%,"Free (A+,AU,BH,CL,CO,D,E,IL,JO,KR,MA,OM,P,PA,P...",15%,"[{'columns': ['general'], 'marker': '1', 'valu...",None,None,NaN
9,0101.90,1,Other:,None,[],,,,[],,,NaN
